In [1]:
import numpy as np
import math
from qiskit import IBMQ, Aer
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, assemble, transpile
from qiskit.visualization import plot_histogram
import qiskit.quantum_info as qi
from qiskit.dagcircuit import DAGCircuit
from qiskit.converters import circuit_to_dag

inputString = "10"

circ = QuantumCircuit(2)
if(inputString == "01"):
    circ.x(1)
elif(inputString == "10"):
    circ.x(0)
elif(inputString == "11"):
    circ.x(0)
    circ.x(1)

circ.barrier()
circ.h(0)
circ.cx(0,1)
circ.barrier()
circ.draw()

┌───┐ ░ ┌───┐      ░ 
q_0: ┤ X ├─░─┤ H ├──■───░─
     └───┘ ░ └───┘┌─┴─┐ ░ 
q_1: ──────░──────┤ X ├─░─
           ░      └───┘ ░

In [2]:
circ.qasm(filename='bellState.qasm')

'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2];\nx q[0];\nbarrier q[0],q[1];\nh q[0];\ncx q[0],q[1];\nbarrier q[0],q[1];\n'

In [3]:
def qft_rotations(circuit, n):
    """Performs qft on the first n qubits in circuit (without swaps)"""
    if n == 0:
        return circuit
    n -= 1
    circuit.h(n)
    for qubit in range(n):
        circuit.cp(math.pi/2**(n-qubit), qubit, n)
    # At the end of our function, we call the same function again on
    # the next qubits (we reduced n by one earlier in the function)
    qft_rotations(circuit, n)
    
def swap_registers(circuit, n):
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    return circuit

def qftCircuit(circuit, n):
    """QFT on the first n qubits in circuit"""
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

In [4]:
n = 3

qc3 = QuantumCircuit(n,n)
qftCircuit(qc3,n)
qc3.draw()

┌───┐   
q_0: ──────■──────────────────────■───────┤ H ├─X─
           │                ┌───┐ │P(π/2) └───┘ │ 
q_1: ──────┼────────■───────┤ H ├─■─────────────┼─
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │ 
q_2: ┤ H ├─■────────■───────────────────────────X─
     └───┘                                        
c: 3/═════════════════════════════════════════════

In [5]:
qc3.qasm(filename='qft3.qasm')

'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[3];\ncreg c[3];\nh q[2];\ncp(pi/4) q[0],q[2];\ncp(pi/2) q[1],q[2];\nh q[1];\ncp(pi/2) q[0],q[1];\nh q[0];\nswap q[0],q[2];\n'